## Import libraries

In [17]:
import torch
import torchvision
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
from torchvision import datasets
import os

## Train Autoencoder

In [2]:
if not os.path.exists('./vae_img'):
    os.mkdir('./vae_img')


def to_img(x):
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x


num_epochs = 100
batch_size = 128
learning_rate = 1e-3

img_transform = transforms.Compose([
    transforms.ToTensor()
    # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = MNIST('./data', transform=img_transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if torch.cuda.is_available():
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return F.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar


model = VAE()
if torch.cuda.is_available():
    model.cuda()

reconstruction_function = nn.MSELoss(size_average=False)


def loss_function(recon_x, x, mu, logvar):
    """
    recon_x: generating images
    x: origin images
    mu: latent mean
    logvar: latent log variance
    """
    BCE = reconstruction_function(recon_x, x)  # mse loss
    # loss = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)
    # KL divergence
    return BCE + KLD


optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(dataloader):
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img)
        if torch.cuda.is_available():
            img = img.cuda()
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(img)
        loss = loss_function(recon_batch, img, mu, logvar)
        loss.backward()
        train_loss += loss.data
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch,
                batch_idx * len(img),
                len(dataloader.dataset), 100. * batch_idx / len(dataloader),
                loss.data / len(img)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(dataloader.dataset)))
    if epoch % 10 == 0:
        save = to_img(recon_batch.cpu().data)
        save_image(save, './vae_img/image_{}.png'.format(epoch))

torch.save(model.state_dict(), './vae.pth')

Train Epoch: 0 [0/60000 (0%)]	Loss: 183.676865
Train Epoch: 0 [12800/60000 (21%)]	Loss: 45.187099
Train Epoch: 0 [25600/60000 (43%)]	Loss: 42.488605
Train Epoch: 0 [38400/60000 (64%)]	Loss: 38.989422
Train Epoch: 0 [51200/60000 (85%)]	Loss: 37.602478
====> Epoch: 0 Average loss: 45.2671
Train Epoch: 1 [0/60000 (0%)]	Loss: 37.402729
Train Epoch: 1 [12800/60000 (21%)]	Loss: 34.094231
Train Epoch: 1 [25600/60000 (43%)]	Loss: 36.393234
Train Epoch: 1 [38400/60000 (64%)]	Loss: 35.329041
Train Epoch: 1 [51200/60000 (85%)]	Loss: 33.767498
====> Epoch: 1 Average loss: 34.8616
Train Epoch: 2 [0/60000 (0%)]	Loss: 33.174458
Train Epoch: 2 [12800/60000 (21%)]	Loss: 33.061054
Train Epoch: 2 [25600/60000 (43%)]	Loss: 34.152504
Train Epoch: 2 [38400/60000 (64%)]	Loss: 33.052860
Train Epoch: 2 [51200/60000 (85%)]	Loss: 34.183891
====> Epoch: 2 Average loss: 33.0455
Train Epoch: 3 [0/60000 (0%)]	Loss: 33.032677
Train Epoch: 3 [12800/60000 (21%)]	Loss: 31.276497
Train Epoch: 3 [25600/60000 (43%)]	Loss: 

Train Epoch: 28 [25600/60000 (43%)]	Loss: 28.570789
Train Epoch: 28 [38400/60000 (64%)]	Loss: 30.629684
Train Epoch: 28 [51200/60000 (85%)]	Loss: 28.481743
====> Epoch: 28 Average loss: 29.2502
Train Epoch: 29 [0/60000 (0%)]	Loss: 30.174021
Train Epoch: 29 [12800/60000 (21%)]	Loss: 29.047501
Train Epoch: 29 [25600/60000 (43%)]	Loss: 29.471247
Train Epoch: 29 [38400/60000 (64%)]	Loss: 27.916105
Train Epoch: 29 [51200/60000 (85%)]	Loss: 29.007961
====> Epoch: 29 Average loss: 29.2216
Train Epoch: 30 [0/60000 (0%)]	Loss: 28.800220
Train Epoch: 30 [12800/60000 (21%)]	Loss: 27.507565
Train Epoch: 30 [25600/60000 (43%)]	Loss: 29.240309
Train Epoch: 30 [38400/60000 (64%)]	Loss: 29.175335
Train Epoch: 30 [51200/60000 (85%)]	Loss: 30.258394
====> Epoch: 30 Average loss: 29.1814
Train Epoch: 31 [0/60000 (0%)]	Loss: 28.845753
Train Epoch: 31 [12800/60000 (21%)]	Loss: 30.306904
Train Epoch: 31 [25600/60000 (43%)]	Loss: 30.310234
Train Epoch: 31 [38400/60000 (64%)]	Loss: 29.328053
Train Epoch: 31 [

Train Epoch: 56 [25600/60000 (43%)]	Loss: 29.247900
Train Epoch: 56 [38400/60000 (64%)]	Loss: 29.587086
Train Epoch: 56 [51200/60000 (85%)]	Loss: 27.743217
====> Epoch: 56 Average loss: 28.6577
Train Epoch: 57 [0/60000 (0%)]	Loss: 29.061302
Train Epoch: 57 [12800/60000 (21%)]	Loss: 28.293606
Train Epoch: 57 [25600/60000 (43%)]	Loss: 29.515631
Train Epoch: 57 [38400/60000 (64%)]	Loss: 27.644247
Train Epoch: 57 [51200/60000 (85%)]	Loss: 28.170116
====> Epoch: 57 Average loss: 28.6252
Train Epoch: 58 [0/60000 (0%)]	Loss: 27.568607
Train Epoch: 58 [12800/60000 (21%)]	Loss: 28.772818
Train Epoch: 58 [25600/60000 (43%)]	Loss: 28.738129
Train Epoch: 58 [38400/60000 (64%)]	Loss: 28.690214
Train Epoch: 58 [51200/60000 (85%)]	Loss: 29.499655
====> Epoch: 58 Average loss: 28.6294
Train Epoch: 59 [0/60000 (0%)]	Loss: 28.631235
Train Epoch: 59 [12800/60000 (21%)]	Loss: 28.710907
Train Epoch: 59 [25600/60000 (43%)]	Loss: 28.085670
Train Epoch: 59 [38400/60000 (64%)]	Loss: 27.647476
Train Epoch: 59 [

Train Epoch: 84 [25600/60000 (43%)]	Loss: 27.627066
Train Epoch: 84 [38400/60000 (64%)]	Loss: 28.466825
Train Epoch: 84 [51200/60000 (85%)]	Loss: 27.368299
====> Epoch: 84 Average loss: 28.3363
Train Epoch: 85 [0/60000 (0%)]	Loss: 29.110487
Train Epoch: 85 [12800/60000 (21%)]	Loss: 26.514534
Train Epoch: 85 [25600/60000 (43%)]	Loss: 28.300285
Train Epoch: 85 [38400/60000 (64%)]	Loss: 29.096092
Train Epoch: 85 [51200/60000 (85%)]	Loss: 27.519148
====> Epoch: 85 Average loss: 28.3272
Train Epoch: 86 [0/60000 (0%)]	Loss: 29.128960
Train Epoch: 86 [12800/60000 (21%)]	Loss: 28.217701
Train Epoch: 86 [25600/60000 (43%)]	Loss: 28.445143
Train Epoch: 86 [38400/60000 (64%)]	Loss: 28.766052
Train Epoch: 86 [51200/60000 (85%)]	Loss: 28.905764
====> Epoch: 86 Average loss: 28.3298
Train Epoch: 87 [0/60000 (0%)]	Loss: 28.341631
Train Epoch: 87 [12800/60000 (21%)]	Loss: 26.555923
Train Epoch: 87 [25600/60000 (43%)]	Loss: 28.292265
Train Epoch: 87 [38400/60000 (64%)]	Loss: 28.520332
Train Epoch: 87 [

## Use trained model to predict images

In [28]:
IMG_PATH = "D:\\University-Work\CPS040\Main\CPS040-Thesis\Main\Autoencoder\Simple Autoencoder 2\\vae_img\\"
min_img_size = 32
trans = transforms.Compose([transforms.Resize(min_img_size),
                                         transforms.CenterCrop(32),
                                         transforms.ToTensor(),
                                         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                              std=[0.229, 0.224, 0.225])])

# # Picture dataset.
# classify_dataset = datasets.ImageFolder(root=IMG_PATH, transform=trans)
# # Create custom random sampler class to iter over dataloader.
# classify_loader = DataLoader(dataset=classify_dataset, batch_size=1, shuffle=True, num_workers=5)

model = torch.load('./vae.pth')

# if cuda is available, move the model to the GPU
# if torch.cuda.is_available():
#     model.cuda()


In [33]:
def to_img(x):
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x
img_transform = transforms.Compose([
    transforms.ToTensor()
    # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = MNIST('./data', transform=img_transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# for batch_idx, data in enumerate(dataloader):
#     data

img, _ = data
img = img.view(img.size(0), -1)
img = Variable(img)
if torch.cuda.is_available():
    img = img.cuda()
    
model = torch.load('./vae.pth')
model(data)

TypeError: 'collections.OrderedDict' object is not callable